In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from tqdm import tqdm
import gensim 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.externals import joblib

/home/citigo/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [6]:
df = pd.read_csv("Data_train.csv", encoding = "utf-8")
df.head()

,Unnamed: 0,product,level_1,level_2,target,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,1541.0,áo len vàng,Thời trang - Phụ kiện,Thời trang - Phụ kiện không phân biệt giới tính,Áo Nam/Nữ,NaN,NaN,count
1,4041.0,Áo len rớt vai 211 NQ,Thời trang - Phụ kiện,Thời trang nữ,"Áo cardigan, áo len, áo hoodie nữ",NaN,NaN,0
2,4081.0,ao len lo lo pha mau,Thời trang - Phụ kiện,Thời trang - Phụ kiện không phân biệt giới tính,Áo Nam/Nữ,NaN,Thực phẩm - Đồ uống khác,2
3,7421.0,ÁO TD LEN MẮT CƯỜM HỒNG,Thời trang - Phụ kiện,Thời trang nữ,"Áo cardigan, áo len, áo hoodie nữ",NaN,Đồ dùng phòng bếp khác,5
4,7841.0,Áo len sweet cổ viền hoa lụa BG nhí H2043,Thời trang - Phụ kiện,Thời trang nữ,"Áo cardigan, áo len, áo hoodie nữ",NaN,Đồ hộp khác,7


In [7]:
data = df.drop(['Unnamed: 0', 'level_2','target', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7'] , axis=1)

data.head()

,product,level_1
0,áo len vàng,Thời trang - Phụ kiện
1,Áo len rớt vai 211 NQ,Thời trang - Phụ kiện
2,ao len lo lo pha mau,Thời trang - Phụ kiện
3,ÁO TD LEN MẮT CƯỜM HỒNG,Thời trang - Phụ kiện
4,Áo len sweet cổ viền hoa lụa BG nhí H2043,Thời trang - Phụ kiện


In [8]:
sents = data['product']
sents = sents.str.lower()
X = []
for sent in sents:
    split = ViTokenizer.tokenize( str(sent))
    print (split)
    X.append(split)
#splitwords =  [ViTokenizer.tokenize( str(sent) ) for sent in sents] 
#print(splitwords)

áo len vàng
áo len rớt vai 211 nq
ao len lo lo pha_mau
áo td len mắt cườm hồng
áo len sweet cổ viền hoa lụa bg nhí h2043
áo len nữ cổ tròn_w
len pm verca
len lông vai lưới
áo vàng nỉ bông
áo_khoác len ngắn cánh dơi
áo dạ khuy có mũ
khoac da ca nu
bbn bra đùi 19395 - 460
ao lot 1339
áo ngực nosbra 103
áo kiểu yumiji nút ngực rút nền cam sp14168
1951 - bộ syzanne vega ren dây_chéo - mận
quần qs596m + . keden
áo_phông đen loewe
áo thun tăm
áo thun dài tay hm trễ vai tk t9
set áo peplum đen tay lưới
áo_phông moses queen hình cây dừa california
pull trắng mickey
áo td lệch chữ mạ trắng
korea - áo thun from rộng in hình đủ hình thú
áo croptop cúc lưng
áo nhung hình môi
áo bbd bèo kẻ
áo thô tay bèo màu xanh size 100
somi 1n2187bt4 / sn4v
áo sơmi giãn tdài hvy
áo sơ_mi gân_cổ nhúng
smi trắng_v . anh
áo sơ_mi trắng_tay dài nút trắng
sơ_mi polo chấm bi xanh đen
an107 - áo kiểu sơ_mi cao_cấp có cổ phối hoa_văn_thân có hoa tiết sang_trọng
sơ_mi zara trắng vnxk_dư
áo sơmi sọc linen yth nhập bt
bộ s

bộ 3 nồi sharp nắp inox cao_cấp dùng được bếp từ
mishio - bộ 5 nồi inox đáy từ mk82
nồi lẩu 2 ngăn size 32cm , chất_liệu inox 304
nồi chiên không dầu perfect usa 4,4 l - gla - 609 - hàng chính hãng
nồi thủy_tinh luminarc lavi015 - c6313 1.5 l
nồi nhôm chống dính lock & lock baum marble 20cm
nồi đất cao_cấp dong hwa tucbeghi 1l g - 702
nồi đất tráng men nắp thủy_tinh cao_cấp 0,5 l
bộ kim_thêu đan có lỗ xỏ chỉ to - 9 chiếc
bộ 50 nút màu tự_nhiên 4 lỗ
kim_khâu
chỉ khâu đen
miếng_vá
chỉ khâu các màu
cúc nhỏ
mũi dệt
khóa kéo
đèn trợ sáng , đèn pha led l4 ngắn siêu sáng 2019
đui_đèn thông_minh cảm_ứng chuyển_động thân_nhiệt tpe sl01
chấn_lưu pha led 20 w giá cho 2 chiếc
bộ 10 đui_đèn treo chống nước e27
bộ nguồn , chấn_lưu , tăng phô cy 70w màu vàng cho đèn led
bóng đèn compact doublewing 36w điện_quang - điện việt
bộ đèn_pin gắn xe_đạp 5 bóng led cho học_sinh và sinh_viên ( đen )
chuột bóng đèn huỳnh_quang 1m2 / 6 tấc_hàn quốc_loosy ( tắc te )
bóng đèn sợi đốt đuôi xoáy e12 110v 10w
quần sh

In [10]:
t = data['level_1']
y = np.array(t)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer
model = Pipeline([('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight="balanced")))])

In [13]:
parameters = {'vectorizer__ngram_range': [(1, 1), (1, 2),(2,2)],
               'tfidf__use_idf': (True, False)}

In [14]:
gs_clf_svm = GridSearchCV(model, parameters, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(X, y)
print(gs_clf_svm.best_score_)
print(gs_clf_svm.best_params_)

/home/citigo/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


0.8159166976571216
{'tfidf__use_idf': True, 'vectorizer__ngram_range': (1, 2)}


In [15]:
model = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight="balanced")))])

In [16]:
model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabula...)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 OneVsRestClassifier(estimator=LinearSVC(C=1.0,
                                                         class_weight='balanced',
     

In [17]:
pred = model.predict(X_test)

In [18]:
accuracy_score(y_test, pred)

0.8736059479553904

In [20]:
joblib.dump(model, 'model.pkl', compress=1)

['model.pkl']

In [21]:
model = joblib.load('model.pkl')